In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

***크롬드라이버 실행(자기 크롬 버전에 맞게 설치후 test.ipynb와 같은 경로에 두기)***

In [ ]:
driver = webdriver.Chrome('chromedriver')
url_list = ["36528615", "31317229", "1269259731", "1779562085", "33040151", "1675801137", "1384361907", "1372982703", "1681990192", "1971062401"]


***클릭할 버튼이 없을때까지 스크롤 내림 (일단 임시로 100지정)***

In [2]:
from selenium.webdriver.common.by import By


In [ ]:
import time
import pandas as pd
userlink = pd.DataFrame()
for _url in url_list[0]:
    URL = f"https://m.place.naver.com/restaurant/{_url}/review/visitor"  
    driver.get(URL)
    time.sleep(2.5)
    for _ in range(100):
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(0.5)
        try: driver.find_element(By.CLASS_NAME, 'fvwqf').click()
        except: pass
        time.sleep(0.1)
    
    link_list = driver.find_elements(By.CLASS_NAME, "Hazns")
    link_list = [link.get_attribute("href") for link in link_list]
    user_list = driver.find_elements(By.CLASS_NAME, "sBWyy")
    user_list = [user.text for user in user_list]
    review_time_list = driver.find_elements(By.CLASS_NAME, "sb8UA")
    review_time_list = [reviews.text for reviews in review_time_list]
    length = min(len(user_list), len(link_list), len(review_time_list))
    userlink2 = pd.DataFrame({'user' : user_list[:length], 'link' : link_list[:length], 'review_time' : review_time_list[:length]}, dtype = str)
    userlink = pd.concat([userlink, userlink2], axis = 0, sort=False)

In [5]:
# iframe 아이디를 반환해줌 
iframes = driver.find_elements(By.CSS_SELECTOR, 'iframe')
for iframe in iframes:
    print(iframe.get_attribute('id'))


searchIframe


In [ ]:
# driver.switch_to.parent_frame() # 아래 코드를 이용해서 iframe갔을때 빠져나오기 위한 코드 (현재는 의미없음)

**식당 리스트 뽑기위한 코드**

In [23]:
import time
import pandas as pd
driver = webdriver.Chrome('chromedriver')
URL = "https://map.naver.com/v5/search/%EA%B0%95%EB%82%A8%EA%B5%AC%20%EC%9D%BC%EC%9B%90%EB%8F%99%20%EC%9D%8C%EC%8B%9D%EC%A0%90?c=14145501.7747606,4507886.3437289,14,0,0,0,dh"  
driver.get(URL)

In [24]:
driver.switch_to.frame('searchIframe') #iframe 으로 이동하기 위한 코드
restaurants_list = pd.DataFrame()

In [25]:
tag_list = driver.find_elements(By.CLASS_NAME, "MVx6e") # 식당 종류 리스트
tag_list = [user.text.split("\n")[-1] for user in tag_list]
tag_list

['4.54리뷰 881TV2TV생생정보',
 '4.36리뷰 999+',
 '곧 영업 시작리뷰 316서울 강남구 일원동',
 '곧 영업 시작리뷰 228서울 강남구 일원동',
 '4.52리뷰 763',
 '영업 전리뷰 744밥이 맛있는 일원동 초밥집',
 '4.77리뷰 319',
 '4.42리뷰 298',
 '4.4리뷰 288',
 '영업 중리뷰 70서울 강남구 일원동']

In [26]:
from tqdm import tqdm
epoch = 6 # 6이 최대인듯
for _ in tqdm(range(epoch)):
    click_list = driver.find_elements(By.CLASS_NAME, "tzwk0")
    click_list[0].click() #아무거나 클릭을 해야 end키가 먹혀서 임의로 아무 클래스 클릭한거임
    for _ in range(10):
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END) # end키를 눌러서 아래까지 로딩
    time.sleep(2)
    click_list = driver.find_elements(By.CLASS_NAME, "tzwk0") # 식당 리스트
    name_list = driver.find_elements(By.CLASS_NAME, "TYaxT") # 식당 이름 리스트
    name_list = [user.text for user in name_list]
    tag_list = driver.find_elements(By.CLASS_NAME, "KCMnt") # 식당 종류 리스트
    tag_list = [user.text for user in tag_list]
    review_list = driver.find_elements(By.CLASS_NAME, "MVx6e") # 리뷰 개수 리스트
    review_list = [user.text.split("\n")[-1] for user in review_list]
    url_list = []
    for restaurant_link in click_list:
        restaurant_link.click()
        url_list.append(driver.current_url)
        time.sleep(0.5)
    restaurants_list2 = pd.DataFrame({'restaurant' : name_list, 'tag' : tag_list, 'url' : url_list, 'review' : review_list}, dtype = str)
    restaurants_list = pd.concat([restaurants_list, restaurants_list2], axis = 0, sort=False)
    driver.find_elements(By.CLASS_NAME, "yUtES")[1].click() #다음페이지 클릭
    time.sleep(2)

100%|██████████| 6/6 [05:22<00:00, 53.75s/it]


In [27]:
restaurants_list.to_csv('river_behind5.csv', index=False)

In [28]:
data = pd.read_csv('./save_csv/river_behind_concat.csv')
url_list = list(data['url'].values)
url_list[:5]

[1675303081, 1867823297, 38969614, 895457986, 11700226]

In [30]:
len(url_list)

3071

In [29]:
userlink = pd.DataFrame()

In [32]:
from tqdm import tqdm
#url_list = ["36528615", "31317229", "1269259731", "1779562085", "33040151", "1675801137", "1384361907", "1372982703", "1681990192", "1971062401"]
current_status = 2100

for _url in tqdm(url_list[2100:2500]):
    if current_status % 50 == 0: #크롬 드라이버 가끔 이상해지는거때문에 50번마다 재실행
        driver = webdriver.Chrome('chromedriver')
    URL = f"https://m.place.naver.com/restaurant/{_url}/review/visitor"  
    driver.get(URL)
    time.sleep(2.5)
    prev_height = driver.execute_script("return document. body.scrollHeight")
    while True:
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(0.5)
        try: driver.find_element(By.CLASS_NAME, 'fvwqf').click()
        except: pass
        current_height = driver.execute_script("return document. body.scrollHeight")
        if current_height == prev_height: break
        prev_height = current_height
        time.sleep(0.1)
    
    link_list = driver.find_elements(By.CLASS_NAME, "Hazns")
    link_list = [link.get_attribute("href") for link in link_list]
    user_list = driver.find_elements(By.CLASS_NAME, "sBWyy")
    user_list = [user.text for user in user_list]
    review_time_list = driver.find_elements(By.CLASS_NAME, "sb8UA")
    review_time_list = [reviews.text for reviews in review_time_list]
    length = min(len(user_list), len(link_list), len(review_time_list))
    userlink2 = pd.DataFrame({'user' : user_list[:length], 'link' : link_list[:length], 'review_time' : review_time_list[:length]}, dtype = str)
    userlink = pd.concat([userlink, userlink2], axis = 0, sort=False)
    current_status += 1

 23%|██▎       | 92/400 [28:37<1:31:41, 17.86s/it]